### Overview: 
- This script identifies stocks exhibiting sustained relative strength against market indices. 
- It specifically filters for equities breaking out above the upper 50-day Bollinger Band (2σ limit).<br>
<b>Trading Rules <br></b>
<b>Entry (Buy):</b>
- Stock must maintain higher weekly strength than the index for 4 consecutive weeks.
- Closing price must exceed the upper Bollinger Band (50-period MA, 2 standard deviations).<br>
<b>Exit (Sell):</b>
- Close below the 100-day Moving Average (MA).
- Price drops below a trailing stop defined as $2 \times ATR(13)$.

In [1]:
import pandas as pa
from GetFreshMarketData import *

In [2]:
from datetime import datetime, timedelta

# Get current date and time
now = datetime.now()
now.weekday()

5

In [4]:
end_date = datetime.today()
start_date = end_date - timedelta(days = 365 * 3)
past_six_months = end_date - timedelta(days = 90)
file_cr = 40000000
weekly_relative_performace = 4
buy_bb_band_period = 50
buy_bb_band_sigma = 2
sell_sma = 100
sell_atr_period = 13
sell_atr_mult = 2

all_traded_stock = pd.read_csv(INDEX_DIR_CONST/'all_traded_stock.csv',low_memory=False)
all_traded_stock.date = all_traded_stock.date.apply(lambda x : datetime.strptime(x,'%Y-%m-%d'))
all_traded_stock = all_traded_stock.loc[((all_traded_stock.date <= end_date) & 
                                        (all_traded_stock.date >= past_six_months) &
                                        (all_traded_stock.total_value >=file_cr)),:]
all_traded_stock = all_traded_stock.groupby('symbol').agg({'date':len}).reset_index()
candidate_symbol = all_traded_stock.loc[all_traded_stock.date==all_traded_stock.date.max(),'symbol']

In [5]:
progress_bar = tqdm(INDEX_TICKERS_DF.iterrows(), desc="Downloading historical data", total=len(INDEX_TICKERS_DF.NSE_SYMBOL))

relative_performace_df = pd.DataFrame(columns =['index_name','roc'])
index_contituents_df = pd.DataFrame(columns = ['symbol','index_name'])
weekly_index_roc = {}
all_index_dict = {}
for _, row in progress_bar:
    symbol = row['NSE_SYMBOL']
    fl_name = row['INDEX_NAME']
    index  = pd.read_csv(STOCK_DIR/f'{symbol}.csv')
    index_constituents = pd.read_csv(INDEX_DIR_CONST/f'{fl_name}.csv')
    index.date = pd.to_datetime(index.date)
    index = index.loc[index.date <= end_date,:]
    index = index.set_index('date')
    
    index = index.sort_index()
    
    index_constituents['index_name'] = symbol
    index_contituents_df = pd.concat([index_contituents_df,index_constituents[['symbol','index_name']]])
    
    weekly_index_close = index.resample('W-MON').agg({'close': 'last'})
    weekly_index_close = weekly_index_close.sort_index()
    weekly_index_close = weekly_index_close.ffill()
    weekly_performace_close = weekly_index_close.pct_change(1)
    

    
    if relative_performace_df.empty:
        relative_performace_df = pd.DataFrame({'index_name':[symbol],
                                                        'roc':weekly_performace_close.tail(1).values[0]})
    else:
        relative_performace_df = pd.concat([relative_performace_df,pd.DataFrame({'index_name':[symbol],
                                                        'roc':weekly_performace_close.tail(1).values[0]})])
    
    weekly_index_roc[symbol] = weekly_performace_close
    all_index_dict[symbol] = index


relative_performace_df = relative_performace_df.sort_values(by='roc')

In [6]:
def calculate_bollinger_bands(close: pd.Series, period: int = buy_bb_band_period, sigma: int = buy_bb_band_sigma) -> pd.DataFrame:
                """Calculate Bollinger Bands"""
                rolling_mean = close.rolling(window=period).mean()
                rolling_std = close.rolling(window=period).std()
                upper_band = rolling_mean + (rolling_std * sigma)
                lower_band = rolling_mean - (rolling_std * sigma)
                return pd.DataFrame({
                    'upper_band': upper_band,
                    'lower_band': lower_band
                }, index=close.index)

In [7]:
pbar = tqdm(candidate_symbol,desc = "Processing for ",total=len(candidate_symbol))
final_stocks = pd.DataFrame()
for symbol in pbar:
    pbar.set_description(f'Processing for {symbol}')
    file = STOCK_DIR/f'{symbol}.csv'
    if file.exists():
        df = pd.read_csv(file, low_memory=False)
        df.date = df.date.apply(lambda x : datetime.strptime(x,'%Y-%m-%d'))
        df = df.loc[df.date <= end_date,:]
        df = df.set_index('date')
        df = df.sort_index()

        weekly_stock_close = df.resample('W-MON').agg({'close': 'last'})
        weekly_stock_close = weekly_stock_close.sort_index()
        weekly_stock_close = weekly_stock_close.ffill()
        weekly_stock_performance = weekly_stock_close.pct_change(1)

        relative_performace_count = 0
        indexes_beaten = []
        mannual_check = []
        for index_name,index_weekly_performace in weekly_index_roc.items():
            try:
                relative_performace = (weekly_stock_performance.tail(weekly_relative_performace) > index_weekly_performace.tail(weekly_relative_performace)).sum()
                if relative_performace.values[0] >= weekly_relative_performace:
                    relative_performace_count +=1
                    indexes_beaten.append(index_name)
            except:
                mannual_check.append(symbol, index_name)

        
        # if relative_performace_count >= 6:
        #     print(symbol, relative_performace_count,indexes_beaten)
            
        
        if df['close'].shape[0]>50 and weekly_stock_performance.tail(1).values[0] >= relative_performace_df.roc.iloc[-1] and relative_performace_count >= 6:
            bb = calculate_bollinger_bands(df['close'])
            latest_close = df['close'].iloc[-1]
            latest_upper = bb['upper_band'].iloc[-1]
            if pd.notna(latest_upper) and latest_close > latest_upper:
                temp = pd.DataFrame({'symbol': symbol,
                                    'weekly_return': weekly_stock_performance.tail(1).values[0],
                                    'latest_close': latest_close,
                                    'upper_bb': latest_upper,
                                    'bb_breakout': latest_close / latest_upper - 1,
                                    'indexes_beaten':','.join(indexes_beaten),
                                    'indexes_beaten_count': relative_performace_count
                                    })
                final_stocks = pd.concat([final_stocks, temp])
if not final_stocks.empty:
    final_stocks = final_stocks.sort_values(by='weekly_return', ascending=False)

print(mannual_check)

Processing for ZYDUSLIFE: 100%|██████████| 505/505 [00:36<00:00, 13.73it/s] 

[]


In [8]:
final_stocks.to_csv(TEMP/'final_stock.csv')
final_stocks.head(20)

,symbol,weekly_return,latest_close,upper_bb,bb_breakout,indexes_beaten,indexes_beaten_count
0,IIFLCAPS,0.143268,377.85,369.962423,0.021320,"NIFTY 500,NIFTY 100,NIFTY AUTO,NIFTY BANK,NIFT...",9
0,SANDUMA,0.140009,264.71,244.442027,0.082915,"NIFTY 500,NIFTY 100,NIFTY MIDCAP 100,NIFTY SML...",22
0,IDBI,0.130011,114.73,106.632071,0.075943,"NIFTY 500,NIFTY 100,NIFTY MIDCAP 100,NIFTY SML...",20
0,FORCEMOT,0.120258,21230.00,19854.900336,0.069257,"NIFTY 100,NIFTY AUTO,NIFTY BANK,NIFTY PHARMA,N...",8
0,HINDCOPPER,0.111704,541.90,488.921594,0.108358,"NIFTY 500,NIFTY 100,NIFTY MIDCAP 100,NIFTY SML...",22
0,NATIONALUM,0.097707,330.30,314.745442,0.049419,"NIFTY 500,NIFTY 100,NIFTY MIDCAP 100,NIFTY SML...",21
0,GRAPHITE,0.096302,659.70,637.328800,0.035102,"NIFTY 500,NIFTY 100,NIFTY MIDCAP 100,NIFTY SML...",21
0,SANSERA,0.093768,1871.00,1822.434477,0.026649,"NIFTY 500,NIFTY 100,NIFTY MIDCAP 100,NIFTY AUT...",13
0,ASHOKLEY,0.079236,188.78,185.378464,0.018349,"NIFTY MIDCAP 100,NIFTY PHARMA,NIFTY IT,NIFTY C...",6
0,KARURVYSYA,0.070264,272.50,262.856635,0.036687,"NIFTY 500,NIFTY 100,NIFTY AUTO,NIFTY BANK,NIFT...",10


In [20]:
# check for stock which are giving sell signal
stocks_holding = ['BAJFINANCE','EICHERMOT','MUTHOOTFIN','BHARTIARTL','INDIANB','TECH','HINDZINC']
pbar = tqdm(stocks_holding, desc = "Processing for",total=len(stocks_holding))

sell_signal_stocks = pd.DataFrame()
for symbol in pbar:
    pbar.set_description(f'Processing for {symbol}')
    file = STOCK_DIR/f'{symbol}.csv'
    if file.exists():
        df = pd.read_csv(file, low_memory=False)
        df.date = df.date.apply(lambda x : datetime.strptime(x,'%Y-%m-%d'))
        df = df.set_index('date')
        df = df.sort_index()        
        if len(df) < max(sell_sma, sell_atr_period + 1):
            raise Exception(f'historical less than {max(sell_sma, sell_atr_period + 1)} for {symbol}')
        df['SMA100'] = df['close'].rolling(sell_sma).mean()

        high, low, close = df['high'], df['low'], df['close']
        prev_close = close.shift(1)
        tr1 = high - low
        tr2 = abs(high - prev_close)
        tr3 = abs(low - prev_close)
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        df['ATR'] = tr.rolling(sell_atr_period).mean()
        # Latest conditions
        latest, prev = df.iloc[-5:-1,:], df.iloc[-5:-1,:]
        below_sma = latest['close'] < latest['SMA100']
        big_drop = latest['close'] < (prev['close'] - sell_atr_mult * latest['ATR'])
        if below_sma.sum()>0 or big_drop.sum()>0:
            if sell_signal_stocks.empty:
                sell_signal_stocks = pd.DataFrame({'symbol':[symbol],'close':latest['close'].iloc[0],'below_sma':below_sma.sum(),'big_drop':big_drop.sum()})
            else:
                sell_signal_stocks = pd.concat([sell_signal_stocks, 
                                                pd.DataFrame({'symbol':[symbol],'close':latest['close'].iloc[0],'below_sma':below_sma.sum(),'big_drop':big_drop.sum()})])
    else:
        raise Exception(f'historical data not found for {symbol}')
    


Processing for HINDZINC: 100%|██████████| 7/7 [00:00<00:00, 16.23it/s]  


In [21]:
sell_signal_stocks

,symbol,close,below_sma,big_drop
0,BAJFINANCE,998.0,3,0
